In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

In [2]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []


In [3]:
import glob
import os
import random

# Load image files from the dataset
image_files = [f for f in glob.glob(r'D:/my-files/20-50/20-50/Gender-Detection-master/Gender-Detection-master/gender_dataset_face' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)


In [4]:
# converting images to arrays and labelling the categories
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] # C:\Files\gender_dataset_face\woman\face_1162.jpg
    if label == "woman":
        label = 1
                                                                                  
    else:
        label = 0
        
    labels.append([label]) # [[1], [0], [0],...]

In [5]:
# Pre-processing with float32
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

# Check unique labels
print("Unique train labels:", np.unique(trainY))
print("Unique test labels:", np.unique(testY))

# Find the number of classes based on the unique labels
num_classes = np.max(trainY) + 1  # Ensure that num_classes matches your label range

# Convert labels to categorical
trainY = to_categorical(trainY, num_classes=num_classes)
testY = to_categorical(testY, num_classes=num_classes)

# Verify shape of trainY and testY
print("TrainY shape:", trainY.shape)
print("TestY shape:", testY.shape)



Unique train labels: [0 1]
Unique test labels: [0 1]
TrainY shape: (1845, 2)
TestY shape: (462, 2)


In [7]:
# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [8]:
# define model
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model


In [9]:
# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

C:\Users\Test\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
from keras.optimizers import Adam

# Set learning rate and epochs
learning_rate = 0.001  # Replace with your desired learning rate
epochs = 500  # Replace with your number of epochs

# Compile the model
opt = Adam(learning_rate=learning_rate)  # Remove decay
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Example model for 30 classes
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Change this to 30 classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
H = model.fit(
    aug.flow(trainX, trainY, batch_size=batch_size),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // batch_size,
    epochs=epochs,
    verbose=1
)


Epoch 1/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 277ms/step - accuracy: 0.6007 - loss: 1.9475 - val_accuracy: 0.7771 - val_loss: 0.4895
Epoch 2/500
 1/28 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.7500 - loss: 0.4654

c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7500 - loss: 0.4654 - val_accuracy: 0.7727 - val_loss: 0.4743
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 265ms/step - accuracy: 0.7345 - loss: 0.5173 - val_accuracy: 0.8052 - val_loss: 0.4412
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7547 - loss: 0.4528 - val_accuracy: 0.8268 - val_loss: 0.3908
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.8202 - loss: 0.4068 - val_accuracy: 0.8377 - val_loss: 0.3595
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8438 - loss: 0.4560 - val_accuracy: 0.8420 - val_loss: 0.3544
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 264ms/step - accuracy: 0.8074 - loss: 0.4301 - val_accuracy: 0.8658 - val_loss: 0.3122
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7969 - loss: 0.4927 - val_accuracy: 0.8528 - val_loss: 0.3248
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 267ms/step - accuracy: 0.8224 - loss: 0.3894 - val_accuracy: 0.8463 - v

In [18]:
# save the model to disk
model.save('gender.h5')